In [ ]:
%%capture
!pip install gradio torch efficientnet-pytorch plotly pillow matplotlib ipywidgets

In [ ]:
import sys
sys.path.append('.')

from efficient_net import EfficientNetTrainer
from preprocessor import ODIRImageProcessor

In [ ]:
config = {
    'model_name': 'efficientnet-b4',
    'num_classes': 8,
    'image_size': 512,
}
MODEL_PATH = "ODIR-2019/results/efficientnet-b4-odir-2019-pmg.pth"  # Change this to your model path
trainer = EfficientNetTrainer(config)
trainer.load_model(path=MODEL_PATH)
processor = ODIRImageProcessor()

## SANITY CHECK
Here we check whether our model is able to predict correctly

In [ ]:
trainer.predict_single_image_path(image_path="ODIR-2019/YOLO/processed_512g_merged/test/diabetes/26_left.jpg")

Check for image that has not been processed, using our preprocessor

In [ ]:
trainer.predict_single_image_path(image_path="ODIR-2019/YOLO/preprocessed/test/diabetes/26_left.jpg", 
                             preprocessor=processor.load_image)


In [ ]:
import gradio as gr
import cv2
import numpy as np
from PIL import Image
# Let's assume your updated function now takes a NumPy array:
ODIR_CLASS_DESCRIPTIONS = {
    'normal': 'Healthy fundus with no apparent pathologies',
    'diabetes': 'Diabetic Retinopathy - presence of microaneurysms, hemorrhages, hard exudates, cotton wool spots, or neovascularization',
    'glaucoma': 'Glaucoma - optic nerve damage with increased cup-to-disc ratio, retinal nerve fiber layer defects',
    'cataract': 'Cataract - lens opacity visible through fundus image',
    'ageing': 'Age-related Macular Degeneration - drusen, pigmentary changes, geographic atrophy, or neovascularization in macular region',
    'hypertension': 'Hypertensive Retinopathy - arteriovenous nicking, copper/silver wiring, flame-shaped hemorrhages',
    'myopia': 'Pathological Myopia - tessellated fundus, peripapillary atrophy, posterior staphyloma',
    'other': 'Other retinal conditions including vein occlusion, retinal detachment, tumors, etc.'
}

def diagnosis_wrapper(input_array):
    if input_array is None:
        # Return empty/default values to the outputs to prevent a crash
        return None, {}, "No image provided. Please upload a scan."
    # 1. Handle Color Space
    # Gradio provides RGB. If your model/OpenCV logic expects BGR:
    # input_bgr = cv2.cvtColor(input_array, cv2.COLOR_RGB2BGR)
    img_bgr=cv2.cvtColor(input_array, cv2.COLOR_RGB2BGR)
    img_bgr=processor.preprocess_image(img_bgr)
    img_rgb=cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img_rgb)
    # 2. Run your prediction on the array
    # (Update your predict function to accept the array instead of a path)
    results = trainer.predict_single_image(img)
    
    # 3. Format results for Gradio Label
    confidences = {label: float(conf) for label, conf in results['predictions']}
    predicted_class = results["predicted_class"]
    predicted_conf = round(results["confidence"]*100)
    comment = ODIR_CLASS_DESCRIPTIONS.get(predicted_class,"")
    summary=f"The model predict with {predicted_conf}% confidence that the eye is likely {predicted_class}\n{comment}"
    # 4. Return the image array itself and the labels
    return img_rgb, confidences, summary

# Define the Interface
with gr.Blocks() as demo:
    gr.Markdown("## 👁️ Rapid Eye Analysis")
    
    with gr.Row():
        # INPUT: Set type="numpy" to pass the actual pixel array
        with gr.Column():
            img_input = gr.Image(type="numpy", label="Input Image", width=512, height=512)
        with gr.Column():
            summary = gr.Textbox(label="Summary")    
            label_output = gr.Label(num_top_classes=5)
        with gr.Column():
            img_output = gr.Image(label="Processed Image", width=512, height=512)

    # Link the input change directly to the function for real-time feel
    img_input.change(
        fn=diagnosis_wrapper, 
        inputs=img_input, 
        outputs=[img_output, label_output, summary]
    )

demo.launch()